In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
# import sys
# import os
# py_file_location = "/content/drive/MyDrive/liver-imaging-analysis/engine"
# sys.path.append(os.path.abspath(py_file_location))
# !pip install monai
# !pip install -U scikit-image

In [ ]:
# !pip install torchvision
# !pip install torch
# !pip install matplotlib
# !pip install monai
# !pip install nibabel
# !pip install -U scikit-learn scipy matplotlib
# !pip install -U numpy 
# !pip install tensorboard

In [1]:
import unet
import nibabel
import engine
import torch
import json
from torch import nn
import diceloss
import deeplab

#Run on GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

#A class that adds Engine methods to the model
class NeuralNetwork(deeplab.DeepLab,engine.Engine):
    def __init__(self):
        engine.Engine.__init__(
            self,
            device=device,
            loss=diceloss.DiceLoss(),
            optimizer= config['optimizer']['name'],
            metrics=['dice_score','loss'],
            training_data_path=config['dataset']['training'],
            testing_data_path=config['dataset']['testing'],
            transformation_flag=True,
            data_size=config['transformation_size'],
            batchsize=config['batch_size'],
            train_valid_split=0,
        )        
        deeplab.DeepLab.__init__(self)#,device=device)

#Read configurations from JSON File
with open('../config/configs.json') as f:
    config = json.load(f)


#Instance from the Model
model=NeuralNetwork().to(device)
model.optimizer_init(lr=config['optimizer']['learning_rate'])

#Print loss function, optimizer, and metrics
print("Compile Status")
print("-------------------------------")
model.compile_status()

#Print Dataset Shape
print("\nData Status")
print("-------------------------------")
model.data_status()

# #Load Stored Checkpoint
# model.load_checkpoint(config['model checkpoint'])

#Start Fitting
print("\nTraining")
print("-------------------------------")
model.fit2d(
    epochs=config['epochs'],
    evaluation_set=model.test_dataloader,
    evaluate_epochs=1,
    visualize_epochs=1,
    save_flag=False,
    save_path="best_epoch"
)

#Test model on testing set
print("\nTesting")
print("-------------------------------")
model.test2d(model.test_dataloader)

#Predict mask for a single volume
# print("\nPrediction")
# print("-------------------------------")
# ImagePath=config['ImagePath']
# output=model.predict(ImagePath)
# output.shape


Compile Status
-------------------------------
Loss= DiceLoss(
  (normalization): Sigmoid()
) 

Optimizer= Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.01
    maximize: False
    weight_decay: 0
) 

Metrics= ['dice_score', 'loss'] 


Data Status
-------------------------------
Batch Shape of Training Features: (1, 32, 32, 16) torch.float32
Batch Shape of Training Labels: (1, 32, 32, 16) torch.float32

Training
-------------------------------

Epoch 1/5
-------------------------------
Batch: 1/2
Batch: 2/2
Current Loss=0.8539133965969086

Epoch 2/5
-------------------------------
Batch: 1/2
Batch: 2/2
Current Loss=0.8054708540439606

Epoch 3/5
-------------------------------
Batch: 1/2
Batch: 2/2
Current Loss=0.6977079510688782

Epoch 4/5
-------------------------------
Batch: 1/2
Batch: 2/2
Current Loss=0.6235350072383881

Epoch 5/5
-------------------------------
Batch: 1/2
Batch: 2/2
Current Loss=

0.5265876352787018

In [ ]:
model.save_checkpoint("TESTT")
import matplotlib.pyplot as plt
plt.plot(model.totalloss)

In [ ]:
model.evaluate_test()

In [ ]:
#Predict mask for a single volume
import matplotlib.pyplot as plt
print("\nPrediction")
print("-------------------------------")
ImagePath=config['ImagePath']
output=model.predict(ImagePath)
output.shape
plt.imshow(output[0,0,:,:,32].cpu())